In [ ]:
# 依赖导入
import pandas as pd
import ijson
import csv
import os

In [31]:
# 定义数据文件路径变量
data3_df_path = "data/trades_3.csv"
data4_df_path = "data/trades_4.csv"
data10_df_path = "data/trades_10.csv"
data13_df_path = "data/trades_13.csv"
data14_df_path = "data/trades_14.csv"
data18_df_path = "data/trades_18.csv"
data_paths = [
    data3_df_path,
    data4_df_path,
    data10_df_path,
    data13_df_path,
    data14_df_path,
    data18_df_path,
]

broker_5_path = "broker_5/1_batch_write_close_2010-01-01_2025-01-01.json"
tradingHistory_path = "login_tradings"

data文件夹中的数据分析：

In [23]:
for path in data_paths:
    print(f"数据文件路径: {path}")
    df = pd.read_csv(path)
    login_count = df['login'].nunique()
    print(f"login的数量有: {login_count}")

数据文件路径: data/trades_3.csv
login的数量有: 11898
数据文件路径: data/trades_4.csv
login的数量有: 1307
数据文件路径: data/trades_10.csv
login的数量有: 11274
数据文件路径: data/trades_13.csv
login的数量有: 4171
数据文件路径: data/trades_14.csv
login的数量有: 5142
数据文件路径: data/trades_18.csv
login的数量有: 11298


broker_5 json数据预处理：

In [ ]:
# 定义一个生成器函数，用于逐条读取大型JSON数组文件
def stream_json_array(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        for obj in ijson.items(f, "item"):  # 顶层是数组时，'item' 就是一条记录
            yield obj

In [40]:
allLogins = set()

for record in stream_json_array(
    "broker_5/1_batch_write_close_2010-01-01_2025-01-01.json"
):
    login = str(record.get("login")) + "_" + str(record.get("symbol"))
    login_filename = f"{tradingHistory_path}/{login}.csv"
    
    if login is not None and login not in allLogins:
        allLogins.add(login)
        print(f"新增login: {login}, 当前总数: {len(allLogins)}", end='\r', flush=True)
        # 创建一个新的trades csv文件，文件名为login，保存record中的数据
        with open(login_filename, mode="w", newline='', encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=record.keys())
            writer.writeheader()
            writer.writerow(record)
    else:
        with open(login_filename, mode="a", newline='', encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=record.keys())
            writer.writerow(record)

IncompleteJSONError: parse error: premature EOF
                                       
                     (right here) ------^


In [ ]:
# 删除tradingHistory_path中行数小于等于10的csv文件
for filename in os.listdir(tradingHistory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(tradingHistory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            row_count = sum(1 for _ in f)
        if row_count <= 10:
            os.remove(file_path)
            print(f"已删除: {file_path}", end='\r', flush=True)

In [ ]:
# 完成数据处理后，broker_5中json文件被拆分成多个csv文件，保存在tradingHistory_path目录下, 单个产品交易笔数大于十笔的数量是多少
file_count = len([f for f in os.listdir(tradingHistory_path) if f.endswith('.csv')])
print(f"{tradingHistory_path} 文件夹中的csv文件数量为: {file_count}")

login_tradings 文件夹中的csv文件数量为: 66445
